In [1]:
using MCMCChains, Serialization, Plots, StableRNGs
include("misc.jl")
include("convergence_diagnostics.jl");

In [8]:
#run = "290"
chunks = [1,2]
rng = StableRNG(39485); #for randomly sampled trace plots

In [3]:
parameters = return_inferred_parameters();

ESS

In [6]:
for chunk in chunks
    chain = deserialize("outputs/300_$(run)_posterior_samples_chunk$(chunk).jls")
    ndims, nwalkers, nsamples = size(chain)
    my_ess = return_ess(chain, ndims, nwalkers, nsamples)
    serialize("outputs/400_$(run)_ess_chunk$(chunk).jls", my_ess)
    chain = nothing
end

LoadError: InterruptException:

Plot ESS

In [13]:
for chunk in chunks
    ess = deserialize("outputs/400_$(run)_ess_chunk$(chunk).jls")
    n_parameters = length(ess)*2
    nwalkers = 1000
    min_ess = nwalkers*100
    p1 = plot(bar(1:Int(n_parameters/2), ess, color=:turquoise4, label="ess"))
    plot!([0,Int(n_parameters/2)+1],[min_ess, min_ess], color=:black, linewidth=4, label="minimum ess")
    if chunk == 1
        xticks!(1:Int(n_parameters/2), parameters[1:Int(n_parameters/2)])
    elseif chunk == 2
        xticks!(1:Int(n_parameters/2), parameters[Int(n_parameters/2)+1:Int(n_parameters)])
    end
    plot!(dpi=300, size=(10*100, 2*100))
    savefig("outputs/400_convergence_diagnostics/400_$(run)_ess_chunk$(chunk)_plot.png")
end

Trace Plots

chain = deserialize("outputs/300_$(run)_posterior_samples_chunk1.jls")
ndims, nwalkers, nsamples = size(chain)
p = []
for i in 1:ndims
    p_i = trace_plots(rng, chain[i,:,:], nwalkers, nsamples, parameters[i])
    push!(p,p_i)
end
#my_plot = plot(p[3:end]...,layout=(5,5))
#savefig("outputs/400_$(run)_trace_plots_chunk$(chunk).png")